In [1]:
import sys
sys.path.append("../Scripts")

In [2]:
import os, numpy as np, pandas as pd
import text_preprocessing as preprocessing
import feature_extraction as feature_extraction
import baseline_model as baseline
import robust_classic_model as robust_model

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Vectorizadores
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [14]:
path = 'A:/MICHU/Documents/Escuela/CIC/Doctorado/Author profiling/PAN20-Fake-News/Datasets/CSV/'
training = pd.read_csv(f'{path}Train-ES.csv')
test = pd.read_csv(f'{path}Test-ES.csv')
data = [training, test]

In [15]:
# Creating text preprocessing object
prep = preprocessing.Preprocessing(language='spanish')

In [16]:
# Loop for cleaning text data
data_preprocessed = []
for d in data:
    data_prep = prep.main_preprocess(data=d, 
                                    column='tweet', 
                                    tweet=True, 
                                    tweet_tags=True, 
                                    remove_stop_words=None,
                                    lemmatize=False,
                                    translate_emojis=False,
                                    whitelist = "")
    data_preprocessed.append(data_prep)

In [18]:
# Vectorizers
vectorizer = CountVectorizer(ngram_range = (1,1), 
                            lowercase = False)
oth_feats_vectorizer = CountVectorizer(ngram_range = (1,1), 
                                        lowercase = False)

In [23]:
for i, d in enumerate(data_preprocessed):
    features = feature_extraction.FeatureExtraction(data=d,
                                                    text_column='tweet', 
                                                    lemma=True, 
                                                    pos=True, 
                                                    tag=True,                                
                                                    other_features=True,
                                                    vectorizer=vectorizer,
                                                    oth_feats_vectorizer=oth_feats_vectorizer,
                                                    language='es')
    
    # Getting training features
    if i == 0:
        training_vector, training_vocabulary, training_dataframes = features.features()
    
    # Getting training features
    else:
        test_dataframe = features.add_features_to_test()

Lemma: (30000, 32035)
POS: (30000, 16)
TAG: (30000, 15)
Other features: (30000, 625)
Final vector shape: (30000, 32691)


In [60]:
len(training_vector[1]), len(training_vocabulary)

(32691, 32691)

In [31]:
training_dataframes[1]

0             NOUN PRON DET NOUN VERB ADP DET ADJ NOUN ADJ
1        ADJ NOUN VERB DET NOUN SCONJ SCONJ AUX NOUN DE...
2        NOUN ADP NOUN ADJ ADJ CCONJ ADJ ADP VERB SCONJ...
3        NOUN ADJ ADV PRON VERB DET ADJ NOUN ADP DET VE...
4        DET NOUN ADV ADJ ADP DET NOUN SCONJ PRON PRON ...
                               ...                        
29995    DET NOUN PROPN DET NOUN ADP NOUN PRON ADV VERB...
29996    ADV PRON VERB ADV ADP PRON ADP VERB DET NOUN P...
29997    NOUN ADP NOUN ADP NOUN ADP NOUN CCONJ ADJ NOUN...
29998                                    ADV ADP DET PROPN
29999                              NOUN ADP DET NOUN PROPN
Length: 30000, dtype: object

In [26]:
X_train = training_vector
y_train = training['label'].tolist()
test_vectorizer = vectorizer(vocabulary=training_vocabulary, lowercase=False)
X_test = test_vectorizer.transform(test_dataframe['tweet'].tolist())
y_test = test_dataframe['label'].tolist()

TypeError: 'CountVectorizer' object is not callable

In [56]:
a = training_vocabulary
seen = set()
uniq, index = [], []

for i, x in enumerate(a):
    if x not in seen:
        uniq.append(x)
        seen.add(x)
    else:
        index.append(i)

print(seen)
print(uniq)
print(index)

{'nivea', 'merecio', 'socorrismo', 'servicio', 'masturbacion', 'lacrimogena', 'juventud', 'peligroso', 'rail', 'funsionario', 'romanticismo', 'apoquindo', 'gh', 'skies', 'graza', 'boing', 'sicopata', 'barato', 'bibdepinto', 'marquesado', 'mayoritariamente', 'derretir', 'tomatelo', 'high', 'aires', 'upata', 'sentada', 'gaming', 'acertar', 'cinismo', 'publi', 'deflactacion', 'bmw', 'leanlo', 'contaminir', 'sinergia', 'obrador', 'rabal', 'paam', 'purga', 'oopart', 'implo', 'llegado', 'andamos', 'daorje', 'galiza', 'vear', 'consulado', 'grabir', 'cranberries', 'interesantisir', 'brujeria', 'alianxo', 'pentecostal', 'mtb', 'impredecible', 'common', 'tonificarlo', 'reunion', 'deberan', 'brutalmente', 'grabación', 'fase', 'congregar', 'sindicaloid', 'fred', 'amat', 'tammy', 'ojito', 'sacudir', 'quejarno', 'reinsercion', 'estatica', 'stedman', 'clarisimo', 'hiperglucemia', 'reanalisis', 'demoledora', 'hawai', 'uk', 'alimentar', 'diputa', 'filadelfia', 'idental', 'martuchis', 'podcasts', 'seais

In [58]:
flag = 0
ejemplo = training_vector.tolist()
for i, x in enumerate(index):
    if i == 0:
        ejemplo.pop(x)
    else:
        flag-=1
        ejemplo.pop(x+flag)

print(ejemplo)

IndexError: pop index out of range